# import packages

In [1]:
import os
import math    

# Import videos

In [2]:
# import videos

def list_mp4_files(directory):
    # List to store mp4 file names
    mp4_files = []

    # Iterate through files in the given directory
    for filename in os.listdir(directory):
        # Check if file is an mp4
        if filename.endswith('.mp4'):
            mp4_files.append(filename)
    
    return mp4_files


# save video names

# Example usage
directory_path = '/Users/ellasay/Google Drive/My Drive/1.SLEAP/Videos'
mp4_files = list_mp4_files(directory_path)

In [3]:
print(mp4_files)

['day1_ctrl_restricted_wheel.mp4', 'day2_s2_restricted_familiar.mp4', 'day4_s3_unrestricted_novel_diff_sex.mp4', 'day1_unrestricted_familia.mp4', 'day2_s3_unrestricted_familiar.mp4', 'day4_s1_unrestricted_old_novel.mp4', 'day2_s5_unrestricted_novel_restricted_wheel.mp4', 'day4_s4_diff_sex_restricted_wheel.mp4', 'day3_unrestricted_familiar.mp4', 'day4_s2_restricted_novel_diff_sex.mp4', 'day3_restricted_novel.mp4', 'day2_s4_unrestricted_novel.mp4']


# Create bash script to edit video (brightness, contrast)

In [4]:
# make bash script to edit brightness and contrast using ffmpeg

def edit_video(dir, output, mp4_files, brightness=0.0, contrast=1.0): #defaults brightness 0.0 default contrast 1.0
    # Create a bash script file
    script_path = os.path.join(dir, 'edit_videos.sh')
    
    with open(script_path, 'w') as script_file:
        # Write the bash script header
        script_file.write("#!/bin/bash\n\n")

        # Loop through each mp4 file and generate ffmpeg commands
        for video in mp4_files:
            filename = os.path.splitext(video)[0]
            input_path = os.path.join(dir, video)
            output_path = os.path.join(output, f"{filename}_edited.mp4")
            
            # Generate ffmpeg command to adjust brightness and contrast
            ffmpeg_command = f"ffmpeg -i \"{input_path}\" -vf eq=brightness={brightness}:contrast={contrast} \"{output_path}\"\n"
            
            # Write the command to the bash script
            script_file.write(ffmpeg_command)

    # Make the bash script executable
    os.chmod(script_path, 0o755)
    print(f"Bash script created: {script_path}")

# Change the brightness and contrast here
brightness_value = 0.1 
contrast_value = 1.2   

directory_path = "/Users/ellasay/Google Drive/My Drive/1.SLEAP/Videos"
output_directory = "/Users/ellasay/Google Drive/My Drive/1.SLEAP/Videos/Processed"

edit_video(directory_path, output_directory, mp4_files, brightness_value, contrast_value)


Bash script created: /Users/ellasay/Google Drive/My Drive/1.SLEAP/Videos/edit_videos.sh


# Create bash script to run inference on videos 
batched, run using tmux

In [13]:
def create_inference_script(directory, mp4_files, model_path, batch_size=4):
    # Calculate the number of scripts needed
    num_scripts = math.ceil(len(mp4_files) / batch_size)

    # Write the bash scripts
    for script_index in range(num_scripts):
        # Create a bash script file
        script_filename = f"{str(script_index+1).zfill(2)}_inference.bash" #name scripts
        script_path = os.path.join(directory, script_filename)

        with open(script_path, 'w') as script_file:
            # Write the bash script header
            script_file.write("#!/bin/bash\n\n")

            # Write sleap-track commands for a batch of videos
            start_index = script_index * batch_size
            end_index = min(start_index + batch_size, len(mp4_files))

            for i in range(start_index, end_index):
                video_path = os.path.join(directory, mp4_files[i])
                sleap_command = (
                    f"sleap-track \"{video_path}\" "
                    f"-m \"{model_path}\"\n"

                    # Convert .slp files to .h5 files for analysis
                    f"sleap-convert \"{video_path}.predictions.slp\" "
                    f"--format analysis \n"
                )
                # Write the command to the bash script
                script_file.write(sleap_command)

        # Make the bash script executable
        os.chmod(script_path, 0o755)

    print(f"Created {num_scripts} bash scripts in {directory}")

# Example usage
directory_path = '/Users/ellasay/Google Drive/My Drive/1.SLEAP/Videos'
# mp4_files = ['video1.mp4', 'video2.mp4', 'video3.mp4', 'video4.mp4', 'video5.mp4']  # Example list of videos
model_path = '/Users/ellasay/Google Drive/My Drive/1.SLEAP/Training/models/240903_154039.single_instance.n=441'  # Replace with the actual path to your model

# Create bash scripts, each with 4 videos
create_inference_script(directory_path, mp4_files, model_path, batch_size=4)


Created 3 bash scripts in /Users/ellasay/Google Drive/My Drive/1.SLEAP/Videos
